In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import time
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import regex as re
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from spacy.tokenizer import Tokenizer
from spacy.pipeline import EntityRuler
import re
from spacy.tokens import Span
from spacy.util import filter_spans
from spacy.training.example import Example
from spacy.language import Language
from sklearn.model_selection import train_test_split

import json
import random

In [2]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
tokenizer = nlp.tokenizer
all_stopwords = nlp.Defaults.stop_words


C:\Users\Tahia.Tabassum\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
df = pd.read_csv('merged_cats.csv')
equip = pd.read_csv('Equipment.csv')

In [4]:
#cleaning up the dataframe

def clean_notes(data, col_name):
    data[col_name] = data[col_name].str.replace('lc', 'lamp column')
    data[col_name] = data[col_name].str.replace('l/c', 'lamp column')
    data[col_name] = data[col_name].str.replace('sugg', 'suggested')
    data[col_name] = data[col_name].str.replace('rreturn', 'return')
    data[col_name] = data[col_name].str.replace('o/s', 'outside')

    return data

In [5]:
df = clean_notes(df, 'job_notes')

In [6]:
#checks if a string matches a regex pattern

def regex_checker(text, pattern):
    matches = re.finditer(pattern, text)
    for match in matches:
        print (match)


In [7]:
#load json file
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)



In [8]:
#save data to a json file
def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [9]:
#transforms the data into a json structure
#The counter counts the number of unique entities in the data

def structure_data(text, doc_param, counter):
    results=[]
    entities = []
    ignore = ['GPE', 'DATE', 'ADDRESS', 'ORG', 'PERSON', 'FAC', 'TIME', 'NORP', 'MONEY', 'LOC']
    for ent in doc_param.ents:
        if ent.label_ not in ignore:
            entities.append((ent.start_char, ent.end_char, ent.label_))
            counter[ent.label_] = counter.get(ent.label_, 0) + 1
    
    if len(entities) > 0:
        results = [text, {"entities": entities}]
    else:
        results = None
    
    return (results), counter
    


In [10]:
#to remove the QUANTITY entity from the list of default entities

@Language.component("entity_removal")
def entity_removal(doc):
    ents = list(doc.ents)
    for ent in ents:
        print(ent.label_)
        if ent.label_=='QUANTITY':
            ents.remove(ent)
    ents = tuple(ents)
    doc.ents = ents
    return(doc)
Language.component("entity_removal", func=entity_removal)


<function __main__.entity_removal(doc)>

In [11]:
#training the model

def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        nlp.add_pipe("entity_removal", before="ner")
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.2, sgd = optimizer)
            print (losses)
    return (nlp)

In [12]:
#a csv of keywords needed to be labelled as the 'NEW EQUIPMENT' entity if found in the data

ruler = nlp.add_pipe("entity_ruler", before = "ner")
for i in range(len(equip)):    
    ruler.add_patterns([{"label": "NEW EQUIPMENT", "pattern": equip.iloc[i,0]}])

In [13]:
#patterns to identify new entities
patterns =  [{"label": "ADDRESS", "pattern": [" ?outside \d+ ?", " ?\d{1,3} [A-Za-z]+ road\.?",
                                             " ?\d{1,3} [A-Za-z]+ rd\.?"," ?\d{1,3} [A-Za-z]+ hill\.?",
                                              " ?\d{1,3} [A-Za-z]+ Hill\.?", " ?\d{1,3} st\.? [A-Za-z]+",
                                              " ?\d{1,3} saint\.? [A-Za-z]+", "\s\d{1}[A-Za-z]{2}\s|\s\d{1}[A-Za-z]{2}$|^\d{1}[A-Za-z]{2}\s"]},
            {"label": "COLUMN MEASUREMENT", "pattern": [" ?\d{1,5}\.?\d{0,5}n?m column| ?\d{1,5}\.?\d{0,5}N?M column",
                                                       " ?\d{1,3}\.?\d{0,5} column ?| ?\d{1,5}\.?\d{0,5} meter ?[A-Za-z]*"]},
            {"label": "PHONE NUMBER", "pattern": [" ?\d{3} ?\d{7,8}"]}, 
            
            {"label": "TEMPERATURE", "pattern": [" ?\d{1,4}k ?| ?\d{1,4}K ?"]},
            {"label": "POWER", "pattern": [" ?\d{1,4}w| ?\d{1,4}W"]},
            {"label": "LANTERN FEATURE", "pattern": [" ?\d{1,3} led lantern ?"]},
            
            {"label": "LANTERN FEATURE", "pattern": [" ?\d{1,3} led lantern ?"]},
            {"label": "REFERENCE", "pattern": ["(lamp)? ?column \d{1,4} ?"]},
            {"label": "JOB NUMBER", "pattern": ["( ?job( number)?| ?quote( number)?| ?quotation( number)?) ?\d+ ?"]},
            {"label": "CUSTOM_QUANTITY", "pattern": [" ?remove (one|two|three|four|five|six|seven|eight|nine) [a-z]+ ?[a-z]* ?",
                                              " ?replace (one|two|three|four|five|six|seven|eight|nine) [a-z]+\b ?[a-z]*\b ?",
                                             " ?[a-z]* ?[a-z]* damaged x\d{1,3} ?[a-z]* ?",
                                              " ?x?\d{1,4} illuminated (sign)? posts? ?",
                                              " ?x\d{1,4} ?[a-z]* ?",
                                              " ?[a-z]* ?[a-z]* \d* x\d{1,3} ?[a-z]* ?",
                                              " ?supply (one|two|three|four|five|six|seven|eight|nine) [a-z]+ ?[a-z]* ?",
                                              " ?\d{1,4} ?[a-z]+ to upgrade$ ?"                 
                                             ]}
     ]
                                              

In [14]:
#splitting the dataset
train_df, test_df = train_test_split(df, test_size=0.3)


In [15]:
patterns[0]

{'label': 'ADDRESS',
 'pattern': [' ?outside \\d+ ?',
  ' ?\\d{1,3} [A-Za-z]+ road\\.?',
  ' ?\\d{1,3} [A-Za-z]+ rd\\.?',
  ' ?\\d{1,3} [A-Za-z]+ hill\\.?',
  ' ?\\d{1,3} [A-Za-z]+ Hill\\.?',
  ' ?\\d{1,3} st\\.? [A-Za-z]+',
  ' ?\\d{1,3} saint\\.? [A-Za-z]+',
  '\\s\\d{1}[A-Za-z]{2}\\s|\\s\\d{1}[A-Za-z]{2}$|^\\d{1}[A-Za-z]{2}\\s']}

In [16]:
entity_text = []
train = []
counter = {}

entity_json=[]
#list of new entities
#nlp.add_pipe("entity_removal", before="ner")

for i in range(len(train_df)):
    doc = nlp(train_df.iloc[i, 3])
    
    ents = list(doc.ents)
    for ent in ents:
        if ent.label_=='QUANTITY':
            ents.remove(ent)
    ents = tuple(ents)
    doc.ents = ents
    
    original_ents = list(doc.ents)

    new_ents = []

    #Identifying new multiple-word entities using regex patterns
    for m in range(len(patterns)):
        for n in range(len(patterns[m]['pattern'])):
            for match in re.finditer(patterns[m]['pattern'][n], doc.text):
                start, end = match.span()
                span = doc.char_span(start, end, alignment_mode = 'expand')
                if span is not None:
                    #appending start char, end char, and text of entity
                    new_ents.append((span.start, span.end, span.text))
                    #print(span.text, patterns[m]['label'])
                
        for ent in new_ents:
            start, end, name = ent
            per_ent = Span(doc, start, end, label=patterns[m]['label'])
            original_ents.append(per_ent)    
   
    #prioritizing the matches        
    filtered = filter_spans(original_ents)
    #print(filtered)
    #updating entities
    doc.ents = filtered 
    small_entities =[]

    for ent in doc.ents:
        entity_text.append((ent.text, ent.label_))
    x = [train_df.iloc[i,3], {'entities': entity_text}]
    if len(entity_text)>0:
        entity_json.append(x)
    
    results, counter = structure_data(train_df.iloc[i, 3], doc, counter)
    if results != None:
        train.append(results)
        


    

In [17]:
print (len(train))
save_data("training_data.json", train)


299


In [18]:
save_data("labels_training_data.json", entity_json)


In [19]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [20]:
TRAIN_DATA = load_data("training_data.json")
TRAIN_DATA = trim_entity_spans(TRAIN_DATA)
random.shuffle(TRAIN_DATA)
nlp = train_spacy(TRAIN_DATA, 10)



Starting iteration 0
{'ner': 1499.8445741933665}
Starting iteration 1
{'ner': 700.0904885224621}
Starting iteration 2
{'ner': 490.8532871622607}
Starting iteration 3
{'ner': 437.68283963226963}
Starting iteration 4
{'ner': 363.7619856078855}
Starting iteration 5
{'ner': 306.57576515176913}
Starting iteration 6
{'ner': 277.88810187166524}
Starting iteration 7
{'ner': 357.5757717234614}
Starting iteration 8
{'ner': 237.30689356397855}
Starting iteration 9
{'ner': 223.35312854362647}


In [21]:
#generating the test set from the trained model
small_entities =[]
for i in range(len(test_df)):
    entities = []
    doc = nlp(test_df.iloc[i,3])
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    results = [test_df.iloc[i,3], {'entities': entities}]
    if len(entities)>0:
        small_entities.append(results)

In [22]:
small_entities



[['supply and fit  back light shield  to the street light',
  {'entities': [('shield', 'NEW EQUIPMENT')]}],
 ['lamp column 39 outside 196 queenstown road sw8 3nr\nlamp column 111 queenstown road \nsupply tower for camera  cycle cameras inspection on 15 february 2021 at 2.00pm (site contact safety shield global jon guest m: 07494876195);\n\ninvoice cost code',
  {'entities': [('lamp column 39', 'REFERENCE'),
    ('lamp column 111', 'REFERENCE'),
    ('shield', 'NEW EQUIPMENT')]}],
 ['replace 6m column. w008499.',
  {'entities': [('6m column', 'NEW EQUIPMENT')]}],
 ['perm disconn 2x keep left boll base fit new nal sockets and nal k/l weebols.; number the asset;  b013 and b14',
  {'entities': [('2x', 'CARDINAL'), ('keep left boll', 'NEW EQUIPMENT')]}],
 ['replace damage bollard shell /stem; supply and install new haldo bollard shells ecco;  number the asset',
  {'entities': [('bollard shell /stem', 'NEW EQUIPMENT'),
    ('haldo bollard shells', 'NEW EQUIPMENT')]}],
 ['(1)install socket on

In [23]:
save_data("test_results.json", small_entities)